In [1]:
import numpy as np
from pupil_labs.realtime_api.simple import discover_one_device

import time

from datetime import datetime
import matplotlib.pyplot as plt
import cv2

import nest_asyncio
nest_asyncio.apply()

In [2]:
device = discover_one_device()

print(f"Phone IP address: {device.phone_ip}")
print(f"Phone name: {device.phone_name}")
print(f"Battery level: {device.battery_level_percent}%")
print(f"Free storage: {device.memory_num_free_bytes / 1024**3:.1f} GB")
print(f"Serial number of connected module: {device.module_serial}")

Phone IP address: 192.168.1.172
Phone name: Neon Companion
Battery level: 55%
Free storage: 213.8 GB
Serial number of connected module: 619453


In [10]:
recording_id = device.recording_start()
print(f"Started recording with id {recording_id}")

time.sleep(5)

device.recording_stop_and_save()

Started recording with id 179bef0f-5d9b-4ce5-883d-54b9d09383e6


In [3]:
device.recording_start()

print(device.send_event("python test event 1"))

time.sleep(5)

# send event with current timestamp
print(device.send_event("python test event 2", event_timestamp_unix_ns=time.time_ns()))

device.recording_stop_and_save()

Event(name=None recording_id=26ca4aaf-c66d-46f7-b9fc-5f36e1e6f37f timestamp_unix_ns=1706476270542000000 datetime=2024-01-28 22:11:10.542000)
Event(name=None recording_id=327902ee-52c4-4019-a028-99a23ec69bf3 timestamp_unix_ns=1706476275453132057 datetime=2024-01-28 22:11:15.453132)


In [5]:
device.send_event("python test event 2", time.time_ns())

Event(name=None recording_id=None timestamp_unix_ns=1706476290329195068 datetime=2024-01-28 22:11:30.329195)

In [ ]:
scene_sample, gaze_sample = device.receive_matched_scene_video_frame_and_gaze()

scene_image_rgb = cv2.cvtColor(scene_sample.bgr_pixels, cv2.COLOR_BGR2RGB)
plt.imshow(scene_image_rgb)
plt.scatter(gaze_sample.x, gaze_sample.y, s=200, facecolors='none', edgecolors='r')

In [18]:
imu_sample = device.receive_imu_datum()

dt = datetime.fromtimestamp(imu_sample.timestamp_unix_seconds)
print(f"This IMU sample was recorded at {dt}")

print(f"It contains the following data:")

print(f"Head pose:")
print(imu_sample.quaternion)

print(f"Acceleration data:")
print(imu_sample.accel_data)

print(f"Gyro data:")
print(imu_sample.gyro_data)

This IMU sample was recorded at 1970-01-01 01:00:00
It contains the following data:
Head pose:
Quaternion(x=0.0, y=0.0, z=0.0, w=0.0)
Acceleration data:
Data3D(x=0.0, y=0.0, z=0.0)
Gyro data:
Data3D(x=0.0, y=0.0, z=0.0)


In [36]:
calibration = device.get_calibration()

print("Scene camera matrix:")
print(calibration["scene_camera_matrix"][0])
print("\nScene distortion coefficients:")
print(calibration["scene_distortion_coefficients"][0])

print("\nRight camera matrix:")
print(calibration["right_camera_matrix"][0])
print("\nRight distortion coefficients:")
print(calibration["right_distortion_coefficients"][0])

print("\nLeft camera matrix:")
print(calibration["left_camera_matrix"][0])
print("\nLeft distortion coefficients:")
print(calibration["left_distortion_coefficients"][0])

Scene camera matrix:
[[885.15825992   0.         824.12887524]
 [  0.         885.14923875 612.28065252]
 [  0.           0.           1.        ]]

Scene distortion coefficients:
[-0.13053092  0.10917948 -0.0001823  -0.00043524  0.00068462  0.17075814
  0.05227603  0.02661005]

Right camera matrix:
[[139.37418637   0.          99.23202584]
 [  0.         139.35188665  96.19039145]
 [  0.           0.           1.        ]]

Right distortion coefficients:
[ 3.68394045e-02 -1.23725784e-01  1.05156082e-03 -6.51159021e-04
 -6.49958070e-01 -5.22825142e-02  9.73858508e-03 -6.94081020e-01]

Left camera matrix:
[[139.02147562   0.          93.94689674]
 [  0.         139.25351257  96.6396657 ]
 [  0.           0.           1.        ]]

Left distortion coefficients:
[ 0.05170779 -0.11790007 -0.00186199  0.00106443 -0.60876161 -0.04746648
  0.02421155 -0.65088475]


In [35]:
device.receive_gaze_datum()

GazeData(x=694.6571655273438, y=655.5740356445312, worn=True, timestamp_unix_seconds=1706307864.1201196)

In [ ]:
t = []
for i in range(1000):
    st = time.perf_counter()
    device.receive_gaze_datum()
    est = time.perf_counter()
    t.append(est - st)

[np.mean(t), np.median(t)]

[0.005303709819008873, 0.004529576500317489]

Stopping run loop for rtsp://192.168.1.172:8086/?camera=world
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received for 10 seconds: closing
no RTP received fo

In [ ]:
estimate = device.estimate_time_offset()
print(f"Mean time offset: {estimate.time_offset_ms.mean} ms")
print(f"Mean roundtrip duration: {estimate.roundtrip_duration_ms.mean} ms")